<center><h1><b> Dropout Regularization</b></h1></center>



In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ucisonar/sonar.csv


In [26]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras as ks
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

In [4]:
df = pd.read_csv('/kaggle/input/ucisonar/sonar.csv', header = None)

In [5]:
df[:5]

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [8]:
df.isnull().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [9]:
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
count,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,...,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000,208.000000
mean,0.029164,0.038437,0.043832,0.053892,0.075202,0.104570,0.121747,0.134799,0.178003,0.208259,...,0.016069,0.013420,0.010709,0.010941,0.009290,0.008222,0.007820,0.007949,0.007941,0.006507
std,0.022991,0.032960,0.038428,0.046528,0.055552,0.059105,0.061788,0.085152,0.118387,0.134416,...,0.012008,0.009634,0.007060,0.007301,0.007088,0.005736,0.005785,0.006470,0.006181,0.005031
min,0.001500,0.000600,0.001500,0.005800,0.006700,0.010200,0.003300,0.005500,0.007500,0.011300,...,0.000000,0.000800,0.000500,0.001000,0.000600,0.000400,0.000300,0.000300,0.000100,0.000600
25%,0.013350,0.016450,0.018950,0.024375,0.038050,0.067025,0.080900,0.080425,0.097025,0.111275,...,0.008425,0.007275,0.005075,0.005375,0.004150,0.004400,0.003700,0.003600,0.003675,0.003100
50%,0.022800,0.030800,0.034300,0.044050,0.062500,0.092150,0.106950,0.112100,0.152250,0.182400,...,0.013900,0.011400,0.009550,0.009300,0.007500,0.006850,0.005950,0.005800,0.006400,0.005300
75%,0.035550,0.047950,0.057950,0.064500,0.100275,0.134125,0.154000,0.169600,0.233425,0.268700,...,0.020825,0.016725,0.014900,0.014500,0.012100,0.010575,0.010425,0.010350,0.010325,0.008525
max,0.137100,0.233900,0.305900,0.426400,0.401000,0.382300,0.372900,0.459000,0.682800,0.710600,...,0.100400,0.070900,0.039000,0.035200,0.044700,0.039400,0.035500,0.044000,0.036400,0.043900


In [10]:
df.shape

(208, 61)

In [11]:
X = df.drop(60, axis = 1)
y = df[60]

In [14]:
y = pd.get_dummies(y, drop_first = True).astype(int)
y[:3]

,R
0,1
1,1
2,1


In [15]:
y.value_counts()

R
0    111
1     97
Name: count, dtype: int64

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 0)

In [18]:
 model = ks.Sequential([
    ks.Input(shape=(60,)),  # Input layer
    ks.layers.Dense(60, activation='relu'),  # Hidden layer
    ks.layers.Dense(30, activation='relu'),  # Hidden layer
    ks.layers.Dense(15, activation='relu'),  # Hidden layer
    ks.layers.Dense(1, activation='sigmoid')  # Output layer
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=40, batch_size = 8)


Epoch 1/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6004 - loss: 0.6787
Epoch 2/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7007 - loss: 0.6346 
Epoch 3/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7565 - loss: 0.5927 
Epoch 4/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8036 - loss: 0.5484 
Epoch 5/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8331 - loss: 0.5073 
Epoch 6/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8009 - loss: 0.4962 
Epoch 7/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7597 - loss: 0.5098 
Epoch 8/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8262 - loss: 0.4338 
Epoch 9/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8565 - loss: 0.4213 
Epoch 10/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8450 - loss: 0.3609 
Epoch 11/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8684 - loss: 0.3362 
Epoch 12/40
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0

In [19]:
model.evaluate(X_test, y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8686 - loss: 0.2782  


[0.26171761751174927, 0.8809523582458496]

In [20]:
# Model prediction
y_pred = model.predict(X_test)
y_pred[:5]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


array([[0.20844668],
       [0.90055233],
       [0.3232004 ],
       [0.98174524],
       [0.01277025]], dtype=float32)

In [22]:
# Coverting predictions to binary

y_predict = []
for element in y_pred:
    if element > 0.5:
        y_predict.append(1)
    else:
        y_predict.append(0)

In [23]:
y_predict[:10]

[0, 1, 0, 1, 0, 1, 0, 0, 1, 0]

In [28]:
y_test[:10].T

,12,80,33,5,187,83,116,122,90,154
R,1,1,1,1,0,1,0,0,1,0


In [34]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.83      0.95      0.89        21
           1       0.94      0.81      0.87        21

    accuracy                           0.88        42
   macro avg       0.89      0.88      0.88        42
weighted avg       0.89      0.88      0.88        42



## Dropout Implementation

In [32]:
 model1 = ks.Sequential([
    ks.Input(shape=(60,)),  # Input layer
    ks.layers.Dense(60, activation='relu'),  # Hidden layer
    ks.layers.Dropout(0.5),  # The dropout layer to handle overfit
    ks.layers.Dense(30, activation='relu'),  # Hidden layer
    ks.layers.Dropout(0.5),  # The dropout layer to handle overfit
    ks.layers.Dense(15, activation='relu'),  # Hidden layer
    ks.layers.Dropout(0.5),  # The dropout layer to handle overfit
    ks.layers.Dense(1, activation='sigmoid')  # Output layer
])

model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy']) 

model1.fit(X_train, y_train, epochs=100, batch_size = 8)


Epoch 1/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.5275 - loss: 0.7216
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5162 - loss: 0.6786 
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4694 - loss: 0.7283 
Epoch 4/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5389 - loss: 0.7073 
Epoch 5/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4500 - loss: 0.7210 
Epoch 6/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5374 - loss: 0.6849 
Epoch 7/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5019 - loss: 0.6996 
Epoch 8/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5235 - loss: 0.6895 
Epoch 9/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5412 - loss: 0.6685  
Epoch 10/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5102 - loss: 0.6992 
Epoch 11/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5695 - loss: 0.6603 
Epoch 12/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

In [33]:
model1.evaluate(X_test, y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9211 - loss: 0.2837  


[0.2704543173313141, 0.9285714030265808]

In [35]:
# Model prediction
y_pred1 = model1.predict(X_test)
y_pred1[:5]

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


array([[0.23053628],
       [0.80335695],
       [0.21145593],
       [0.9190558 ],
       [0.01117769]], dtype=float32)

In [37]:
# Coverting predictions to binary

y_predict1 = []
for element in y_pred1:
    if element > 0.5:
        y_predict1.append(1)
    else:
        y_predict1.append(0)

In [38]:
y_predict1[:10]

[0, 1, 0, 1, 0, 1, 0, 0, 1, 0]

In [39]:
y_test[:10].T

,12,80,33,5,187,83,116,122,90,154
R,1,1,1,1,0,1,0,0,1,0


In [40]:
print(classification_report(y_test, y_predict1))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93        21
           1       1.00      0.86      0.92        21

    accuracy                           0.93        42
   macro avg       0.94      0.93      0.93        42
weighted avg       0.94      0.93      0.93        42

